In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from kafka import KafkaProducer
from kafka import KafkaConsumer
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
from pprint import pprint
1#. Ascoltare twtter in streaming e volerlo mandare in un topic apache kafka
#2. cosa deve fare il listener quando riceve un nuovo tweet?
#serve un producer per kafka che ci permette di scrivere su un topic
#viene chiamato il metodo onstatus alla ricezione di un tweet passiamo il producer che abbiamo creato durante il cosstruttore dell'oggeto 
#e il producer toppic che è dove andiamo a scrivere
# Declare a Python class (MyListener) that inherit from tweepy.StreamListener
# Then we override the on_status method that will be automatically used once
# Twitter "sends" us a new tweet.
#streamListener sto ereditando i metodi dalla classe StreamListener
class MyListener(StreamListener):
    def __init__(self, producer, producer_topic):
        super().__init__()
        self.producer = producer
        self.producer_topic = producer_topic
#questo è quello che fa il listerner quando arriva un nuovo tweet automaticamente
#creo un dizionario che è la struttura che voglio mandare poi su kafka
    def on_status(self, status):
        tweet = {
          'user_id': status.user.id,
          'username': status.user.name,
          'location' : status.user.location,
          'text': status.text,
          'data' : (status.created_at).isoformat(),
          'hashtags': [],
          'topic' : 'match'
        }
    #entities è un attributo dell'oggetto che mi è ritornato(status)
        if status.entities.get('hashtags') is not None:
            hashtags = status.entities.get('hashtags')
            tweet['hashtags'] = [ h.get('text') for h in hashtags ]
            #invio info su kafka. il topic è quello creato all'inzio e il
            #tweet è quello che abbiamo appena creato
        self.producer.send(topic=self.producer_topic, value=tweet)
        pprint(tweet)
    
    
    
consumer_key = "uRsBm9atK3qIB21oi5P57vUNW"
consumer_secret = "8mLnLMoxjBsqdscP524G0W616HtyrdIKeDvUQcQyfL2oNx1tOE"
access_token = "1194997794177728512-yApf1RHYMRDfXyDvczEMaqa5izCs9Y"
access_token_secret = "m63WYWAf8s1btu5CvDfAcrR3B2EUIL5DJzKAIAf1YRDpK"


# Authenticate with the OAuth protocol
# See: https://en.wikipedia.org/wiki/OAuth
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Get the API object passing the authentication object
api = tweepy.API(auth)

#creo il producer passandogli la lista nodi kafka
producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
#modo in cui voglio trasformare il valore che arriva
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

#creo un'istanza dell'oggetto MyLIstener e passo il producer appena creato e il nome del topic su cui voglio inviare i messaggi
listener = MyListener(producer=producer, producer_topic="match_topic")
# We need an api to stream, so we reuse the api object we create before.
# Thus, we create a Stream object.
#creiamo lo stream e dopo iniziamo ad ascoltare col metodo filter
stream = Stream(auth = api.auth, listener=listener)

# A number of twitter streams are available through Tweepy.
# For more information on the capabilities and limitations of the different
# streams see Twitter Streaming API Documentation.

# In this example we will use filter to stream all tweets containing the word "Milano".
# The track parameter is an array of search terms to stream.
#ascoltiamo e filtriamo
stream.filter(track=['#INTMIL', '#InterMilan','#DerbyDellaMadonnina', '#DerbyMilano'], languages=["it"])
#quando ricevo un nuovo tweet viene chiamato il metodo on_status
